In [14]:
%reload_ext autoreload
%autoreload 2

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

## Series de datos

Creo las series de datos a usar para entrenar y evaluar los modelos

La serie de datos 'dmps' contiene mediciones cada 10 minutos de la función de densidad de concentración en número de partículas $dn_N(D_p) d log(D_p)$ dados 25 diámetros que están espaciados logarítmicamente entre \SI{10}{\nm} y \SI{800}{\nm}. 

In [16]:
from npfd import data

X_train, X_val, y_train, y_val = data.dataset.make_dataset(dataset_name='dmps', test_size=0.2, seed=27)

print(f"Nº total de archivos: {X_train['count'] + X_val['count']}\n\
Nº de archivos de entrnamiento: {X_train['count']}\n\
Nº de archivos de validación: {X_val['count']}")

Nº total de archivos: 1237
Nº de archivos de entrnamiento: 979
Nº de archivos de validación: 258


In [3]:
with open(y_train['mlf']) as fi:
    tmp = fi.read().replace('\n','').split('.')
    count_ne = 0
    count_e = 0

    for line in tmp:
        if line.startswith('*') or line.startswith("\"") or line.startswith("#"):
            pass
        else:
            if len(line.split()) == 3:
                count_ne += 1
            else:
                count_e += 1

    count_e -= 1
    print(f"""
    ENTRENAMIENTO
    Número de días sin evento: {count_ne} ({count_ne/(count_ne+count_e) * 100})\n\
    Número de días con evento: {count_e}  ({count_e/(count_ne+count_e) * 100})\n\
    Número total de días: {count_ne+count_e}""")


    ENTRENAMIENTO
    Número de días sin evento: 847 (86.51685393258427)
    Número de días con evento: 132  (13.48314606741573)
    Número total de días: 979


In [4]:
with open(y_val['mlf']) as fi:
    tmp = fi.read().replace('\n','').split('.')
    count_ne = 0
    count_e = 0

    for line in tmp:
        if line.startswith('*') or line.startswith("\"") or line.startswith("#"):
            pass
        else:
            if len(line.split()) == 3:
                count_ne += 1
            else:
                count_e += 1

    count_e -= 1
    print(f"""
    EVALUACION
    Número de días sin evento: {count_ne} ({count_ne/(count_ne+count_e) * 100})\n\
    Número de días con evento: {count_e}  ({count_e/(count_ne+count_e) * 100})\n\
    Número total de días: {count_ne+count_e}""")


    EVALUACION
    Número de días sin evento: 220 (85.27131782945736)
    Número de días con evento: 38  (14.728682170542637)
    Número total de días: 258


Genero los graficos de los datos a usar con sus etiquetas

In [5]:
from npfd import visualization as viz

# viz.visualize.generate_plots('train', X_train, y_train)
# viz.visualize.generate_plots('test', X_val, y_val)

## HTK

In [5]:
from npfd.models.base import HiddenMarkovModel

vf = 0.1092
mv = 5.0963637929239754e-5


# Inicializo el modelo
model = HiddenMarkovModel()
model.initialize(X_train, variance_floor=vf, minimum_variance=mv)
model.train(X_train, y_train, minimum_variance=mv)

# Editoel modelo NE y entreno
model.edit([f'AT 2 4 0.2 {{ne.transP}}', 
            f'AT 4 2 0.2 {{ne.transP}}',
            f'AT 2 4 0.2 {{e.transP}}', 
            f'AT 4 2 0.2 {{e.transP}}',
           ])
model.train(X_train, y_train, minimum_variance=mv)

# Aumento el número de gaussianas
gaussian_duplication_times = 4

for i in range(1, gaussian_duplication_times+1):
    model.edit([f'MU {2**i} {{*.state[2-4].mix}}'])
    model.train(X_train, y_train, minimum_variance=mv)
    
    results = model.test(X_val, y_val)
    print(f"mfcc: {results['MMC']}")


/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/notebooks/RPIC
Pruning-Off

Pruning-Off

Pruning-Off


Pruning-Off

Pruning-Off

Pruning-Off


Pruning-Off

Pruning-Off

Pruning-Off


====================== HTK Results Analysis =======================
  Date: Tue Sep 14 20:31:59 2021
  Ref : >ocumentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_labels.mlf
  Rec : >gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf
------------------------ Overall Results --------------------------
SENT: %Correct=70.93 [H=183, S=75, N=258]
WORD: %Corr=97.60, Acc=55.09 [H=326, D=8, S=0, I=142, N=334]
------------------------ Confusion Matrix -------------------------
       e   n 
           e  Del [ %c / %e]
   e  34   0    4
  ne   0  292   4
Ins   71  71

mfcc: 0.19381919607069836

Pruning-Off

Pruning-Off

Pruning-Off


====================== HTK Results Analysis =======================
  Date: Tue Sep 14 20:32:08 2021
  Ref : >ocumentos/Facultad/Tesis/models/bdb/data/inte

## Analizo los resultados

In [6]:
!HResults -A -T 1 -f -p  -t -I /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_labels.mlf /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/monophones ../../data/interim/results.mlf > ../../npfd/models/HTK/analisis_PP/listado_alineamientos.txt


In [8]:
!grep "N=  1]" ../../npfd/models/HTK/analisis_PP/listado_alineamientos.txt | wc
!grep "N=  2]" ../../npfd/models/HTK/analisis_PP/listado_alineamientos.txt | wc
!grep "N=  3]" ../../npfd/models/HTK/analisis_PP/listado_alineamientos.txt | wc

    220    2640   15026
      0       0       0
     38     459    2584


In [9]:
! awk -f ../../npfd/models/HTK/analisis_PP/f.awk ../../npfd/models/HTK/analisis_PP/listado_alineamientos.txt

EVENT {0}:18-10-23.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {1}:19-04-24.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {2}:18-12-05.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {3}:17-01-28.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {4}:17-11-15.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {5}:16-11-25.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {6}:15-03-15.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {7}:17-12-03.rec:  100.00(100.00)  [H=   3, D=  0, S=  0, I=  0, N=  3]
EVENT {8}:16-04-06.rec:   33.33( 33.33)  [H=   1, D=  2, S=  0, I=  0, N=  3]
Aligned transcription: /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A/16-04-06.lab vs /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A/16-04-06.rec
 LAB: ne e ne 
 REC:      ne : FALSO NEGATIVO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
EVENT {9}:19

In [145]:
! awk -f ../../npfd/models/HTK/analisis_PP/fnoevent.awk ../../npfd/models/HTK/analisis_PP/listado_alineamientos.txt

NO EVENT {0}:20-04-27.rec:  100.00(100.00)  [H=   1, D=  0, S=  0, I=  0, N=  1]
NO EVENT {1}:20-05-07.rec:  100.00(100.00)  [H=   1, D=  0, S=  0, I=  0, N=  1]
NO EVENT {2}:18-01-13.rec:  100.00(100.00)  [H=   1, D=  0, S=  0, I=  0, N=  1]
NO EVENT {3}:20-06-11.rec:  100.00(100.00)  [H=   1, D=  0, S=  0, I=  0, N=  1]
NO EVENT {4}:17-05-14.rec:  100.00(100.00)  [H=   1, D=  0, S=  0, I=  0, N=  1]
NO EVENT {5}:15-04-21.rec:  100.00(100.00)  [H=   1, D=  0, S=  0, I=  0, N=  1]
NO EVENT {6}:15-09-14.rec:  100.00(100.00)  [H=   1, D=  0, S=  0, I=  0, N=  1]
NO EVENT {7}:20-02-22.rec:  100.00(-100.00)  [H=   1, D=  0, S=  0, I=  2, N=  1]
Aligned transcription: /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A/20-02-22.lab vs /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A/20-02-22.rec
 LAB:      ne 
 REC: ne e ne : FALSO POSITIVO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
NO EVENT {8}:15-08-27.rec:  100.00(100.00)  [H=   1, D=  0, S=  0

In [19]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'sans-serif',
    'font.size': 15,
    'text.usetex': True,
    'pgf.rcfonts': False,
})

plt.rcParams['xtick.bottom'] = False
plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.labeltop'] = True

classes = ['E', 'NE']


cm_obs = [[965, 312],[5939, 30794]]

proportions = [965/38010, 312/38010,5939/38010, 30794/38010] 
proportions = [round(x*100,2) for x in proportions]
trans = cm_obs

tick_marks = np.arange(len(classes))

colors = ['white','white','white','black']

fig,ax = plt.subplots(figsize=(3,3))
ax.imshow(trans, origin='upper', cmap='cividis')
for ((j,i),label), per, c in zip(np.ndenumerate(trans), proportions, colors):
    ax.text(i,j,f'{label}\n({per}%)',color=c,ha='center',va='center',weight="bold")


plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)
ax.set_xlabel('Auromatic')
ax.xaxis.set_label_position('top')
ax.set_ylabel('Manual')

#plt.show()
fig.tight_layout()  
fig.savefig('/home/gfogwil/Documentos/Facultad/RPIC/figures/cm_obs.pdf', dpi=400, bbox_inches='tight') # , dpi=400

In [17]:
cm_day = [[36, 2],[60, 160]]

classes = ['E', 'NE']


proportions = [36/258, 2/258,60/258, 160/258] 
proportions = [round(x*100,2) for x in proportions]

trans = cm_day

tick_marks = np.arange(len(classes))

colors = ['white','white','white','black']
fig,ax = plt.subplots(figsize=(3,3))
ax.imshow(trans, origin='upper', cmap='cividis')
for ((j,i),label), per, c in zip(np.ndenumerate(trans), proportions, colors):
    ax.text(i,j,f'{label}\n({per}%)',color=c,ha='center',va='center',weight="bold")


plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)
ax.set_xlabel('Automatic')
ax.xaxis.set_label_position('top')
ax.set_ylabel('Manual')

#plt.show()
plt.tight_layout(pad=1)  
fig.savefig('/home/gfogwil/Documentos/Facultad/RPIC/figures/cm_day.pdf', dpi=400, bbox_inches='tight')


In [12]:
print(f"""
PV: {results['TP']+61}
NV: {results['TN']+1400}
FP: {results['FP']+150}
FN: {results['FN']+50}
TOT: {results['TP']+results['TN']+results['FP']+results['FN']}
""")


PV: 938
NV: 29406
FP: 7292
FN: 356
TOT: 36331



In [13]:
results

{'SENT_Correct': 73.26,
 'SENT_H': 189.0,
 'SENT_S': 69.0,
 'SENT_N': 258.0,
 'WORD_Corr': 98.2,
 'WORD_Acc': 58.68,
 'WORD_H': 328.0,
 'WORD_D': 6.0,
 'WORD_S': 0.0,
 'WORD_I': 132.0,
 'WORD_N': 334.0,
 'mlf': PosixPath('/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf'),
 'FNR': 0.2586644125105664,
 'TP': 877,
 'TN': 28006,
 'FP': 7142,
 'FN': 306,
 'F1': 0.19061073679634863,
 'MMC': 0.23029828450849243,
 'TPR': 0.7413355874894336,
 'N': 36331}

In [67]:
from npfd import visualization as viz

viz.visualize.generate_plots('results', X_val, y_val, results)

## Grafico de etiquetas

In [13]:
import pathlib
import matplotlib
import matplotlib.pyplot as plt

from npfd.data.htk import read_data

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
cm = matplotlib.colors.ListedColormap(['#2897d4', '#c62828'])

date = '17-12-25'

file = pathlib.Path(f'/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A/{date}')
xlim = [50, 100]


y1 = y_val
y2 = results
_, obs = read_data(file)
file_length = obs.__len__()

l1 = data.labels.mlf_to_dataframe(y1['mlf'], date)
l1.columns = ['l1']
l1 = l1.replace({'ne':0, 'e':1})

l2 = data.labels.mlf_to_dataframe(y2['mlf'], date)
l2.columns = ['l2']
l2 = l2.replace({'ne':0, 'e':1})

f = plt.figure()

size = f.get_size_inches()

ax1 = plt.subplot2grid((12, 12), (0, 0), rowspan=10, colspan=12)
plt.pcolor(obs.values[1::1, 1::1].T, cmap='jet')
plt.clim(0, 4)
ax1.axes.get_xaxis().set_visible(False)
ax1.axes.get_yaxis().set_visible(False)

ax2 = plt.subplot2grid((12, 12), (10, 0), rowspan=2, colspan=12)
ax2.pcolor(l1.T, cmap=cm)
# ax2.axes.get_yaxis().set_visible(False)
# ax2.axes.get_xaxis().set_visible(False)
plt.setp(ax2.get_yticklabels(), visible=False)
ax2.tick_params(axis='y', which='both', left=False)
# ax2.set_ylabel('M', rotation=0, ha='right', va='center', ma='left')
ax2.set_xticks(range(xlim[0], xlim[1]))
ax2.grid(which='major', axis='x')
ax2.tick_params(axis='x', which='both', bottom=False)
plt.setp(ax2.get_xticklabels(), visible=False) 


# ax3 = plt.subplot2grid((14, 12), (12, 0), rowspan=2, colspan=12)
# ax3.pcolor(l2.T, cmap=cm)
# ax3.set_xticks(range(xlim[0], xlim[1]))
# ax3.grid(which='major')
# ax3.tick_params(axis='x', which='both', bottom=False)
# plt.setp(ax3.get_xticklabels(), visible=False) 
# plt.setp(ax3.get_yticklabels(), visible=False)
# ax3.tick_params(axis='y', which='both', left=False)
# ax3.set_ylabel('A', rotation=0, ha='right', va='center', ma='left')

if xlim is not None:
    plt.setp(ax1, xlim=xlim)
    plt.setp(ax2, xlim=xlim)
#     plt.setp(ax3, xlim=xlim)
f.tight_layout(h_pad=0)   

f.savefig('/home/gfogwil/Documentos/Facultad/RPIC/figures/labels.pgf')

## Grafico espectrograma

In [37]:
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as mdates
import numpy as np
from npfd.data.size_distribution import cm3_to_dndlogdp


plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.labelbottom'] = True
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.labeltop'] = False

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
cm = matplotlib.colors.ListedColormap(['#2897d4', '#c62828'])

date = '2017-12-24'

file = pathlib.Path(f'/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/dmps/2017/{date}.csv')
day = pd.read_csv(file, index_col='datetime', parse_dates=['datetime'])

day = day.loc['2017-12-24 06:00:00': '2017-12-24 22:00:00']

fig, ax = plt.subplots()

fig.set_figheight(3)
fig.set_figwidth(8)

im = plt.pcolor(day.index,  # X axis
           [float(r) for r in day.columns],  # Y axis
           np.log10(np.absolute(day + 10)).values[1::1, 1::1].T,  # Z axis
           cmap='jet')

plt.yscale('log')

ax.xaxis.set_major_formatter(mdates.DateFormatter('%H'))

plt.clim(0, 4)
cbar = fig.colorbar(im, ticks=[0, 1, 2, 3, 4])
cbar.ax.set_yticklabels(['1', '10', '100', '1000', '10000'])

cbar.set_label(label=r"$\frac{dN}{dlog(D_p)} [cm^{-1}]$")

plt.ylabel(r"Di\'ametro [nm]")
plt.xlabel('Hora [UTC-03]')
fig.tight_layout(h_pad=0)   
fig.savefig('/home/gfogwil/Documentos/Facultad/Tesis/figures/espectrograma_mbi.pdf')